In [1]:
print("Hello")

Hello


In [5]:
%pip install qafacteval

Defaulting to user installation because normal site-packages is not writeable
  Using cached qafacteval-0.10-py3-none-any.whl (3.3 kB)
  Using cached qaeval-0.1.0-py3-none-any.whl (42 kB)
  Using cached edlib-1.3.9.tar.gz (91 kB)
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached qaeval-0.0.9-py3-none-any.whl (36 kB)
  Using cached qaeval-0.0.8-py3-none-any.whl (36 kB)
  Using cached qaeval-0.0.7-py3-none-any.whl (28 kB)
  Using cached qaeval-0.0.6-py3-none-any.whl (26 kB)
  Using cached qaeval-0.0.5-py3-none-any.whl (26 kB)
  Using cached qaeval-0.0.3-py3-none-any.whl (18 kB)
  Using cached qaeval-0.0.2-py3-none-any.whl (18 kB)
  Using cached qaeval-0.0.1-py3-none-any.whl (18 kB)
INFO: pip is looking at multiple versions of qafacteval to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install qafacteval because these package versions have conflicting dependencies.

The conflict is caused by:
    qaeval 0.1.0

In [4]:
from qafacteval import QAFactEval
kwargs = {"cuda_device": 0, "use_lerc_quip": True, \
        "verbose": True, "generation_batch_size": 32, \
        "answering_batch_size": 32, "lerc_batch_size": 8}

model_folder = "" # path to models downloaded with download_models.sh
metric = QAFactEval(
    lerc_quip_path=f"{model_folder}/quip-512-mocha",
    generation_model_path=f"{model_folder}/generation/model.tar.gz",
    answering_model_dir=f"{model_folder}/answering",
    lerc_model_path=f"{model_folder}/lerc/model.tar.gz",
    lerc_pretrained_model_path=f"{model_folder}/lerc/pretraining.tar.gz",
    **kwargs
)

results = metric.score_batch_qafacteval(["This is a source document"], [["This is a summary."]], return_qa_pairs=True)
score = results[0][0]['qa-eval']['lerc_quip']


ModuleNotFoundError: No module named 'qafacteval'